# Starting Spark Session

In [1]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("local")
         .appName("read-postgres")
         # Add postgres jar
         .config("spark.driver.extraClassPath", "/home/jovyan/work/jars/postgresql-42.4.0.jar")
         .getOrCreate())
sc = spark.sparkContext

## Read Postgres

### Movies Data

In [2]:
# Movies Table; created from the csv we mounted  

df_movies = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://mypostgres:5432/test")
    .option("dbtable", "public.movies")
    .option("user", "airflow")
    .option("password", "airflow")
    .load()
)

In [16]:
# Show Top 10 rows 

df_movies.show(10)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
+-------+--------------------+--------------------+
only showing top 10 rows



### Ratings Data

In [4]:
# Ratings 

df_ratings = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://mypostgres:5432/test")
    .option("dbtable", "public.ratings")
    .option("user", "airflow")
    .option("password", "airflow")
    .load()
)

In [15]:
df_ratings.show(10)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      1|   4.0|2000-07-30 18:45:03|
|     1|      3|   4.0|2000-07-30 18:20:47|
|     1|      6|   4.0|2000-07-30 18:37:04|
|     1|     47|   5.0|2000-07-30 19:03:35|
|     1|     50|   5.0|2000-07-30 18:48:51|
|     1|     70|   3.0|2000-07-30 18:40:00|
|     1|    101|   5.0|2000-07-30 18:14:28|
|     1|    110|   4.0|2000-07-30 18:36:16|
|     1|    151|   5.0|2000-07-30 19:07:21|
|     1|    157|   5.0|2000-07-30 19:08:20|
+------+-------+------+-------------------+
only showing top 10 rows



# Top 10 movies with more ratings

In [5]:
df_movies = df_movies.alias("m")
df_ratings = df_ratings.alias("r")

df_join = df_ratings.join(df_movies, df_ratings.movieId == df_movies.movieId).select("r.*","m.title")

In [6]:
from pyspark.sql import functions as F

df_result = (
    df_join
    .groupBy("title")
    .agg(
        F.count("timestamp").alias("qty_ratings")
        ,F.mean("rating").alias("avg_rating")
    )
    .sort(F.desc("qty_ratings"))
    .limit(10)
)

In [7]:
df_result.coalesce(1).write.format("csv").mode("overwrite").save("/home/jovyan/work/resources/data/output_postgres", header=True)

In [10]:
df_result.show()

+--------------------+-----------+-----------------+
|               title|qty_ratings|       avg_rating|
+--------------------+-----------+-----------------+
| Forrest Gump (1994)|        329|4.164133738601824|
|Shawshank Redempt...|        317|4.429022082018927|
| Pulp Fiction (1994)|        307|4.197068403908795|
|Silence of the La...|        279|4.161290322580645|
|  Matrix, The (1999)|        278|4.192446043165468|
|Star Wars: Episod...|        251|4.231075697211155|
|Jurassic Park (1993)|        238|             3.75|
|   Braveheart (1995)|        237|4.031645569620253|
|Terminator 2: Jud...|        224|3.970982142857143|
|Schindler's List ...|        220|            4.225|
+--------------------+-----------+-----------------+

